In [2]:
import os, sys
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv3 🚀 v9.6.0-3-gb870de5 torch 1.10.0 CPU


Setup complete ✅


In [17]:
data_dir = "snacks"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")
images_train_dir = os.path.join(data_dir, "images_train")
images_val_dir = os.path.join(data_dir, "images_val")
images_test_dir = os.path.join(data_dir, "images_test")
labels_train_dir = os.path.join(data_dir, "labels_train")
labels_val_dir = os.path.join(data_dir, "labels_val")
labels_test_dir = os.path.join(data_dir, "labels_test")

In [4]:
train_annotations = pd.read_csv(os.path.join(data_dir, "annotations-train.csv"))
train_annotations.head()
    

,image_id,x_min,x_max,y_min,y_max,class_name,folder
0,009218ad38ab2010,0.192620,0.729831,0.127606,0.662219,cake,cake
1,009218ad38ab2010,0.802377,1.000000,0.102585,0.405338,ice cream,cake
2,009218ad38ab2010,0.814884,0.970607,0.121768,0.357798,ice cream,cake
3,00c7515143b32051,0.135084,0.904315,0.081735,0.909091,cake,muffin
4,00c7515143b32051,0.155722,0.910569,0.083403,0.893244,muffin,muffin


In [5]:
print(train_annotations.iloc[0][1])

0.19262


In [6]:
val_annotations = pd.read_csv(os.path.join(data_dir, "annotations-val.csv"))
test_annotations = pd.read_csv(os.path.join(data_dir, "annotations-test.csv"))

In [7]:
label_index = {'apple':0, 'strawberry':1, 'popcorn':2, 'ice cream':3, 'waffle':4, 
               'doughnut':5, 'watermelon':6, 'muffin':7, 'juice':8, 'cookie':9, 
               'banana':10, 'grape':11, 'salad':12, 'hot dog':13, 'pineapple':14,
               'candy':15, 'cake':16, 'pretzel':17, 'orange':18, 'carrot':19}

In [21]:
train_data = load_images_with_annotations(train_dir, data_dir + "/annotations-train.csv")

NameError: name 'load_images_with_annotations' is not defined

In [40]:
from shutil import copy

def convert_dataset(data_dir, images_dir, annotations, dst_img_dir, dst_label_dir):
    print(images_dir)
    labels = os.listdir(images_dir)
    print(labels)
    for label in labels:
        if label == '.DS_Store':
            continue
        label_dir = os.path.join(images_dir, label)
        images = os.listdir(label_dir)
        for image in images:
            if image[-4:] != '.jpg':
                continue
            image_index = image[:-4]
            image_bds = get_object_bds(image_index, annotations, dst_label_dir)
            copy(os.path.join(label_dir, image),os.path.join(dst_img_dir, image))


In [67]:
import math
def get_object_bds(image_index, annotations, dst_label_dir):
    # Find the corresponding row(s) in the CSV's dataframe.
    rows = annotations[annotations["image_id"] == image_index]

    img_annotations = []
    for row in rows.itertuples():
        # The CSV file stores the coordinate as numbers between 0 and 1,
        # but Turi wants pixel coordinates in the image.
        xmin = row[2]
        xmax = row[3]
        ymin = row[4]
        ymax = row[5]
        
        # A bounding box in Turi is given by a center coordinate and the
        # width and height, we have them as the four corners of the box.
        width = xmax - xmin
        height = ymax - ymin
        x = xmin + math.floor(width / 2)
        y = ymin + math.floor(height / 2)
        label_id = label_index[row[6]]
#         print(row[6])
        img_annotations.append([label_id, x, y, width, height])
            
        # If there were no annotations for this image, then append a None
        # so that we can filter out those images from the SFrame.
#     print(img_annotations)
    with open(os.path.join(dst_label_dir, image_index) + '.txt', 'w') as output:
        for row in img_annotations:
            s = " ".join(map(str, row))
            output.write(s+'\n')
#     img_annotations = pd.DataFrame(img_annotations)
#     print(img_annotations)
#     img_annotations.to_csv(os.path.join(dst_label_dir, image_index) + '.txt', header=None, index=None, sep=' ', mode='a')


In [68]:
cwd = os.getcwd()
print(cwd)

convert_dataset(data_dir, train_dir, train_annotations, images_train_dir, labels_train_dir)

/Users/ParagonLight/OneDrive/iOS智能开发/yolov3
snacks/train
['apple', 'strawberry', '.DS_Store', 'popcorn', 'ice cream', 'waffle', 'doughnut', 'watermelon', 'muffin', 'juice', 'cookie', 'banana', 'grape', 'salad', 'hot dog', 'pineapple', 'candy', 'cake', 'pretzel', 'orange', 'carrot']


In [69]:
convert_dataset(data_dir, test_dir, test_annotations, images_test_dir, labels_test_dir)

snacks/test
['apple', 'strawberry', '.DS_Store', 'popcorn', 'ice cream', 'waffle', 'doughnut', 'watermelon', 'muffin', 'juice', 'cookie', 'banana', 'grape', 'salad', 'hot dog', 'pineapple', 'candy', 'cake', 'pretzel', 'orange', 'carrot']


In [70]:
convert_dataset(data_dir, val_dir, val_annotations, images_val_dir, labels_val_dir)

snacks/val
['apple', 'strawberry', '.DS_Store', 'popcorn', 'ice cream', 'waffle', 'doughnut', 'watermelon', 'muffin', 'juice', 'cookie', 'banana', 'grape', 'salad', 'hot dog', 'pineapple', 'candy', 'cake', 'pretzel', 'orange', 'carrot']


In [73]:
# Train YOLOv3 on COCO128 for 5 epochs


SyntaxError: invalid syntax (<ipython-input-73-1754b0b62391>, line 2)